In [1]:
%pip install streamlit==1.32.2 \
llama-index==0.10.20 \
llama-index-embeddings-huggingface==0.1.4 \
llama-index-llms-huggingface==0.1.4 \
chainlit \
gradio --quiet

In [2]:
import gradio as gr
import logging
import sys
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings, PromptTemplate, StorageContext, load_index_from_storage
import torch

# Set up logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger()
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

# Log message to indicate the start of the application
logger.info("Starting Chat with the banking AI application")

Settings, PromptTemplate, StorageContext, load_indexbanking_from_storage

# Set up logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger()
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

# Log message to indicate the start of the application
logger.info("Starting Chat with the banking AI application")

# Define system prompt
SYSTEM_PROMPT = """
You are an AI assistant that answers questions in a friendly manner, based on the given source documents. Here are some rules you always follow:
- Generate human-readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Generate professional language.
- Never generate offensive or foul language.
- Replace TSB bank name with Juvo Bank

"""

# Define query wrapper prompt
query_wrapper_prompt = PromptTemplate("[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] ")

logger.info("Defined system and query wrapper prompts")

# Initialize HuggingFaceLLM
llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.25, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.2",
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto",
    tokenizer_kwargs={"max_length": 2048},
    model_kwargs={"torch_dtype": torch.float16}  # Uncomment this if using CUDA to reduce memory usage
)
logger.info("Initialized HuggingFaceLLM")

# Configure settings
Settings.chunk_size = 512
Settings.llm = llm
Settings.embed_model = "local"
logger.info("Configured settings")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.
Some parameters are on the meta device device because they were offloaded to the cpu.


In [3]:
# Load the chat engine
storage_context = StorageContext.from_defaults(persist_dir="/content/drive/MyDrive/colab/embeddings")
index = load_index_from_storage(storage_context)
chat_engine = index.as_chat_engine(chat_mode="condense_question", verbose=True)

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        print(message)
        response = chat_engine.chat(message)
        bot_message = response.response
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5b879e468a74af213e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
